<a href="https://colab.research.google.com/github/MohanKumar2002/Chatbot-using-NLTK-and-Tensorflow/blob/master/Chatbot_with_nltk_and_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [17]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model


lemmatizer = WordNetLemmatizer()

intents = json.loads(open('/content/intents.json').read())

words = pickle.load(open('/content/words.pkl', 'rb'))
classes = pickle.load(open('/content/classes.pkl', 'rb'))
model = load_model('/content/chatbotmodel.h5')

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word)  for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words= clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1

    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda  x:x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list


def get_response(intents_list,intents_json):
    tag= intents_list[0]['intent']
    list_of_intents =intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

print(" Welcome to Cube AI Tech Solutions ")
print("Ask your any query about our Company")
while True:
    message = input("| You: ")
    if message == "bye" or message == "Goodbye":
        ints = predict_class(message)
        res = get_response(ints, intents)
        print("| Bot:", res)
        print("The Program End here!")
        exit()
        break

    else:
        ints = predict_class(message)
        res = get_response(ints, intents)
        print("| Bot:", res)

 Welcome to Cube AI Tech Solutions 
Ask your any query about our Company
| You: course
1/1 [==============================] - 0s 146ms/step
| Bot: Informatics College Pokhara has been in direct partnership with London Metropolitan University, 
UK to provide enviable higher education in IT and Business to students in Pokhara.
For Bachelors Degree in Information Technology we have been offering the specialization in BSc (Hons) Computing.
For Bachelors in Business Administration we have been offering the followings:

1. BBA (Marketing) with International Business 

2. BBA (Accounting & Finance) with International Business

3. BBA (International Business)
| You: bye
1/1 [==============================] - 0s 20ms/step
| Bot: Goodbye
The Program End here!


In [4]:
!pip install langchain_community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.8 MB/s eta 0:00:00


In [12]:
import random
import json
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load intents data
with open('/content/intents.json') as file:
    intents = json.load(file)

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

print("|============= Welcome to College Bot =============|")
print("|======================== Feel Free ==========================|")
print("|============================ To =============================|")
print("|================ Ask your any query about our Company =======|")

def generate_response(input_text):
    # Find the intent that matches the input_text pattern
    for intent in intents['intents']:
        if any(pattern.lower() in input_text.lower() for pattern in intent['patterns']):
            # If a matching pattern is found, use the intent's responses
            return random.choice(intent['responses'])

    # If no pattern matches, use GPT-2 to generate a response
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output = model.generate(input_ids, max_length=150, num_return_sequences=1, no_repeat_ngram_size=2)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

while True:
    user_input = input("| You: ")

    if user_input.lower() in ["bye", "goodbye"]:
        print("| Bot: Goodbye! Have a nice day.")
        break

    # Generate response
    bot_response = generate_response(user_input)
    print("| Bot:", bot_response)


|============= Welcome to Cube AI Tech Solutions =============|
|======================== Feel Free ==========================|
|============================ To =============================|
|================ Ask your any query about our Company =======|
| You: hi
| Bot: Hello!
| You: how are you
| Bot: Sorry, can't understand you
| You: where are you
| Bot: Please give me more info
| You: what courses you offering
| Bot: Please give me more info
| You: what is your name
| Bot: I'm Ribot
| You: what courses are available
| Bot: Informatics College Pokhara has been in direct partnership with London Metropolitan University, 
UK to provide enviable higher education in IT and Business to students in Pokhara.
For Bachelors Degree in Information Technology we have been offering the specialization in BSc (Hons) Computing.
For Bachelors in Business Administration we have been offering the followings:

1. BBA (Marketing) with International Business 

2. BBA (Accounting & Finance) with Internat